In [28]:
import collections

Sample = collections.namedtuple('Sample', ['timestamp', 'gnss_easting', 'gnss_northing', 'odom_drive', 'odom_steer', 'lidar'])

In [31]:
def read_log(file):
  result = []
  for line in file:
    sample = line.split()
    result.append(Sample(float(sample[0]), float(sample[1]), float(sample[2]), float(sample[3]), float(sample[4]), sample[5::]))
  return result

In [35]:
import math
WHEELBASE_LENGTH = 0.1
def parse_odometry(log):
  Pose = collections.namedtuple('Pose', ['x', 'y', 'theta'])
  poses = [Pose(0.0, 0.0, 0.0)]
  prev_samp = log[0]
  for samp in log[1::]:
    # https://thomasfermi.github.io/Algorithms-for-Automated-Driving/Control/BicycleModel.html
    dt = samp.timestamp - prev_samp.timestamp
    poses.append(
      Pose(
        poses[-1].x + samp.odom_drive * dt * math.cos(poses[-1].theta), 
        poses[-1].y + samp.odom_drive * dt * math.sin(poses[-1].theta), 
        poses[-1].theta + samp.odom_drive * dt * math.tan(samp.odom_steer) * WHEELBASE_LENGTH
        )
      )


[Sample(timestamp=0.0, gnss_easting=100.0, gnss_northing=100.0, odom_drive=0.5, odom_steer=0.1, lidar=['1.0', '2.0', '1.5']), Sample(timestamp=1.0, gnss_easting=101.0, gnss_northing=102.0, odom_drive=0.6, odom_steer=0.1, lidar=['1.1', '2.1', '1.5'])]
[Pose(x=0.0, y=0.0, theta=0.0), Pose(x=0.6, y=0.0, theta=0.0060200803251270335)]


In [ ]:
import matplotlib.pyplot as plt
test_log = read_log(open('./test_log.log', 'r'))
print(test_log)
odometry = parse_odometry(test_log)
print(odometry)

# show odo
fig, ax = plt.subplots()